# `mercury` Demos

> `mercury` is an actor-based modelling framework for games on graphs.

This notebook will go over some **examples usages** and how they relate to Phantom and the market making problem.

In [ ]:
import numpy as np
import mercury as me
import networkx as nx
import matplotlib.pyplot as plt

plt.style.use('bmh')

In [ ]:
def plot(fg, figsize=(18, 6)):
    plt.figure(figsize=figsize)

    fg.is_directed = lambda: True
    pos = nx.spring_layout(fg)

    nx.draw_networkx_nodes(fg, pos, node_size=2500, node_color='skyblue', edgecolors='lightgrey', linewidths=4.0)
    nx.draw_networkx_labels(fg, pos, labels={
        n: n for n in fg.nodes
    })

    nx.draw_networkx_edges(
        fg, pos, edgelist=list(fg.edges), nodelist=list(fg.nodes),
        width=0.25, edge_color='#282828', connectionstyle='arc3, rad=0.1',
        arrows=True, arrowstyle='simple', arrowsize=20, min_target_margin=30.0
    )
    nx.draw_networkx_edge_labels(fg, pos, {
        epair: fg[epair[0]][epair[1]]['flow']
        for epair in fg.edges
    }, font_size=14, label_pos=0.3)

    plt.axis('off')

## Bilateral Trading Demo

As a simple first example, let's consider a graph with two vertices: $\mathcal{V} \doteq \{X, Y\}$.

We'll __interpret $X$ and $Y$ as being traders__ who exchange between "the asset" and cash bilaterally.

For now, we'll use the in-built `SimpleSyncActor` which is a barebones actor used for basic tracking.

In [ ]:
x = me.actors.SimpleSyncActor('X')
y = me.actors.SimpleSyncActor('Y')

x, y

Now, any actor must provide two things:
1. A method for __handling incoming requests and (immediate) responses__.
2. A means by which other actors can __view it's publicly available state__; i.e. a façade.

### Views:

Let's take a brief look at the view API:

In [ ]:
x.view(), y.view()

Above we can see that both $X$ and $Y$ provide a `View` object which contains __only partial information that each trader has made available to the world__.

We can also refine this, to provide tailored information for a given adjacent node:

In [ ]:
x.view('Y')

In this case $X$ does not provide any additional information specifically for $Y$.

In the general case, __this can be used to provide specific information to adjacent nodes on the graph that is only available to them__.

### Messages:

And now a brief look at the messaging API:

`mercury` comes with many different message types. For now, we'll just consider the `FlowMessage` type:
1. This allows us to send explicit `Flow` updates from on node to another.

In [ ]:
flow = me.Flow(inv=5.0, cash=-10.0)
res = x.handle_request('Y', me.FlowMessage(flow))

In the code above we __sent a total of 5 inventory units and -10 units of cash from node $Y$ to node $X$__:
1. This corresponds to $Y$ selling 5 units to $X$ at a price of 2 per unit.

The output of this function is a generator object (i.e. iterator). We can convert this to a list:

In [ ]:
res = list(res)
res

In this case, `SimpleSyncActor` reponds with an equal and opposite message to signify that the request was successful.

This informs the sender node to update it's own inventory and cash:

In [ ]:
y.handle_response('X', res[0][1])

__Important__ - this code is lazy and in some cases you _must consume the generator_ to actually process the incoming order!

Anyway, if we now look at `x` and `y`, they now both have non-zero inventory and cash:

In [ ]:
x.inventory, x.cash

In [ ]:
y.inventory, y.cash

### Networks:

Now we introduce a `Network` which provides the scaffold for specifying relationships:

In [ ]:
net = me.Network({
    'X': me.actors.SimpleSyncActor('X'),
    'Y': me.actors.SimpleSyncActor('Y')
})
plot(net.to_flow_graph(), figsize=(18, 4));

In [ ]:
net['X'], net['Y']

Aside from the unfortunate rendering, the diagram above shows the structure of the problem we just specified. Of course, it's a little boring.

Next, we'll __add an edge between the two nodes__ which implies that there is a _trading relationship between $X$ and $Y$_.

In [ ]:
net.add_connection('X', 'Y')
plot(net.to_flow_graph());

There are a few key things to note about the diagram above:
1. The __edges in the graph reflect the trading routes__ that are available to each actor (i.e. between $X$ and $Y$).
2. __Each edge has a `Flow` object__ associated with it:
    1. These _monitor the flow of inventory and cash from one actor to another_.
    2. The `Network` class enforces these to be _strictly zero-sum_ with respect to the two nodes.

As before, we can send messages across the network between actors:

In [ ]:
net.send({
    'Y': {
        'X': [me.FlowMessage(flow)]
    }
})
plot(net.to_flow_graph());

__Note__ - this operation does not immediately send the request. Instead it just __loads it onto a set of queues!__ _See below; the inventory/cash has not changed:_

__<ins>Important</ins>__: `Flow` objects on the edges are _transient/volatile_! They __only contain information about flow from $t_n \to t_{n+1}$__.

In [ ]:
net['X'].inventory, net['X'].cash

In [ ]:
net['Y'].inventory, net['Y'].cash

__To consolidate the (volatile) messages, we must call `Network.resolve`:__

In [ ]:
net.resolve()

In [ ]:
net['X'].inventory, net['X'].cash

In [ ]:
net['Y'].inventory, net['Y'].cash

In [ ]:
plot(net.to_flow_graph());

### Contexts:

In addition to the `network`, `mercury` has a notion of context implemented as the class `Context`.

In [ ]:
ctx_x = net.context_for('X')

These can be created from the original `network` instance (as above), and they contain information about the local network around a given node, such as...

__The ego-node (i.e. central node):__

In [ ]:
ctx_x.actor

__The IDs of each adjacent node:__

In [ ]:
list(ctx_x.neighbour_ids)

__And the (tailored) views of these nodes:__

In [ ]:
ctx_x.views

In other words, the __`Context` objects represent (in an abstraction) all the information available locally to a node.__

Another key feature of the `Context` objects is that they make sending messages very easy:

In [ ]:
flow = me.Flow(inv=1.0, cash=-2.0)

ctx_x.send_request('Y', me.message.FlowMessage(flow))

In [ ]:
plot(net.to_flow_graph());

And again, if we resolve these messages, then we arrive back at zeros along the edges:

In [ ]:
net.resolve()
plot(net.to_flow_graph());

## Market Making Demo

Now we will look at a simple market making problem with the following properties:
1. There is a **single market maker** who trades bilaterally with **two investors**.
2. The investors are not explicitly aware of one another.
3. **Trades can only occur between the market maker and investors independently**.

Crucially, _we do not care about the behaviour of these agents_. We only care about the mechanism of the market itself: "how do trades occur?", "who can see what about the market?", etc...

### Construct the bipartite network:

`mercury` comes with a number of market making problems pre-built. One of them, is the `FullConnectedBipartite` network. This corresponds to a trading network with two sets of actors - market makers and investors - where all nodes in one group are connected to all nodes in the other group.

In [ ]:
net = me.otc.markets.FullyConnectedBipartite(
    {'MM': me.otc.actors.MarketMaker('MM')},
    {
        'INV_1': me.otc.actors.Investor('INV_1'),
        'INV_2': me.otc.actors.Investor('INV_2')
    }
)

In [ ]:
plot(net.to_flow_graph(), figsize=(26, 8))

### Mutators:

As seen above, we are now using `MarketMaker` and `Investor` types. The latter is basically just a `SimpleSyncActor`, but the `MarketMaker` has some special logic:
1. It has __internal memory which tracks the prices it is currently offering__ to buy/sell off its clients.
2. It can __handle incoming requests to buy/sell assets.__
3. Provides a __client-specific view of the liquidity__ being quoted.

The first part of this requires a mechanism to update the internal memory of the market maker - this is done via __mutation__:

In [ ]:
PRICES = np.linspace(10.0, 20.0, 5)

liq = np.zeros((PRICES.shape[0], 2))
liq[-2:(PRICES.shape[0]+1), 0] = np.random.lognormal(size=2)
liq[0:3, 1] = np.random.lognormal(size=3)

net.mutate({
    'MM': [me.otc.mutators.UpdateBooks(PRICES, {
        'INV_1': me.lob.Liquidity(liq),
        'INV_2': me.lob.Liquidity(liq)
    })]
})

It's not hugely important how this work under the hood. The key point is that __we've now updated the internal state of the market maker__, such that:

In [ ]:
print(net['MM'].books['INV_1'].to_liquidity(PRICES))

In [ ]:
net['MM'].view('INV_1')

In [ ]:
print(net['MM'].view('INV_1').book.to_liquidity(PRICES))

_The two examples illustrate that you can either access actor member variables directly, or use the views._

__The former is not good practice.__ It relies on the assumption that the `Actor` is fixed. `View` objects are meant to be public and will be more consistent.

### Buying and selling:

To take advantage of the liquidty now available, we can send some market order requests:

In [ ]:
net.send({
    'INV_1': {
        'MM': [
            me.lob.messages.PlaceMarketOrder.bid(quantity=1)
        ]
    },
    'INV_2': {
        'MM': [
            me.lob.messages.PlaceMarketOrder.ask(quantity=1)
        ]
    }
})

In [ ]:
net.resolve()

In [ ]:
net['INV_1'].inventory, net['INV_1'].cash

In [ ]:
net['INV_2'].inventory, net['INV_2'].cash

In [ ]:
net['MM'].inventory, net['MM'].cash

## Summary

We've covered a number of topics in this notebook. The key takeaway is that there are 3 components to `mercury`:
1. The `Actor` and `View` types.
2. The `Message` and `Mutator` types.
3. The `Network` and `Context` types.

While you are free to handle everything yourself, the `Network` class makes simulating graph-based games trivial.

Hopefully you should have some idea now of what `mercury` can do.

__The next part of the learning process is in learing how to create your own games.__